In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from geocube.api.core import make_geocube
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys

sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [2]:
import sys
import pandas as pd
import geopandas as gpd
from glob import glob
from multiprocessing import Pool, cpu_count

#cpu_count() - 7

In [4]:
def preprocess_active_fires_subprocess(

                self,

                filenames: list,

                year: int,

                aoi_gdf: gpd.GeoDataFrame

            ):

        """

        Subprocess to handle parallel preprocessing.

        filename_pair: list of two filenames with active fires (Alaska, Canada)

        """

        logging.info('Starting preprocess_active_fires_subprocess')

 

        gdf_list = []

        for filename in filenames:

            

            # read csv file

            af_df = pd.read_csv(filename)

            

            # convert to GeoDataFrame

            af_gdf = gpd.GeoDataFrame(

                af_df,

                geometry=gpd.points_from_xy(af_df.longitude, af_df.latitude),

                crs="EPSG:4326"

            )

            gdf_list.append(af_gdf)

            logging.info(f'{Path(filename).stem} - {af_gdf.shape}')

        

        # concatenate dataframes

        full_gdf = gpd.GeoDataFrame(

            pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs

        ).to_crs(self.epsg)

 

        full_gdf = full_gdf.clip(aoi_gdf)

 

        # save output file

        full_gdf.to_file(os.path.join(self.af_output_dir, f'active_fires_{year}.gpkg'))

        return

def preprocess_active_fires(self):

    """

    Preprocess and georeference active fire.

    """

    # create output directory for active fires

    os.makedirs(self.af_output_dir, exist_ok=True)

    logging.info(f'Created {self.af_output_dir}')



    # get active fire filenames

    af_filenames = glob(self.af_regex)

    logging.info(f'Processing {len(af_filenames)} active fire filenames.')



    # iterate over each year

    filename_pairs = []

    for year in range(self.start_year, self.end_year + 1):



        # we are going to get the files for each year using regex

        # we assume each file has the year in the filename

        file_matches = [fi for fi in af_filenames if str(year) in Path(fi).stem]

        if len(file_matches) == 0:

            logging.info(f'No active fires found for {year}, skipping year.')

            continue

        else:

            filename_pairs.append(file_matches)



    # Distribute a file per processor

    p = Pool(processes=cpu_count())

    p.starmap(

        self.preprocess_active_fires_subprocess,

        zip(

            filename_pairs,

            range(self.start_year, self.end_year + 1),

            repeat(self.aoi_gdf)

        )

    )

    p.close()

    p.join()


    logging.info(f'Saved output to {self.af_output_dir}.')


    return

In [10]:
### Tess tries some stuff

# tmp = pd.DataFrame
# tmp.af_output_dir = "/projects/old_shared/fire_weather_vis/Lightning_analysis/test_para/"
# tmp.
# #preprocess_active_fires()



In [11]:
# csvs_regex = '/projects/2023_lightning_data/*.raw'

# def read_gdf(filename):
#     gdf = gpd.read_file(filename, crs='epsg:4326')
#     return gdf


# def main():

#     csvs_list = glob(csvs_regex)
#     print(len(csvs_list))

#     with Pool(processes=cpu_count()*2) as p:
#         result = p.map(read_gdf, csvs_list)
#     gdf = pd.concat(result)
#     print(gdf.shape)
    
# if __name__ == "__main__":
#     sys.exit(main())

TypeError: 'module' object is not callable